In [1]:
from typing import Dict, Optional, Any, List, Self, Literal, ClassVar, Union
from uuid import UUID, uuid4
from pydantic import BaseModel, Field, model_validator, computed_field, field_validator
from enum import Enum
from dnd.values import ModifiableValue, AdvantageStatus, CriticalStatus, AutoHitStatus, StaticValue,NumericalModifier, ContextualValue

from dnd.blocks import (
    BaseBlock, AbilityScores, SavingThrowSet, Health, 
    Equipped, Speed, ActionEconomy,SkillSet,SkillName
)
from dnd.values import ModifiableValue
from dnd.modifiers import (
    NumericalModifier, DamageType, ResistanceStatus, 
    ContextAwareCondition, BaseObject
)

from dnd.equipment import (
    Armor, Weapon, Shield, BodyArmor, Gauntlets, Greaves,
    Boots, Amulet, Ring, Cloak, Helmet, BodyPart
)
from dnd.dice import Dice, RollType, DiceRoll

In [2]:
from dnd.modifiers import (
    BaseObject,
    naming_callable,
    NumericalModifier,
    AdvantageModifier,
    CriticalModifier,
    AutoHitModifier,
    AdvantageStatus,
    CriticalStatus,
    AutoHitStatus,
    Size,
    DamageType,
    SizeModifier,
    DamageTypeModifier,
    ContextualNumericalModifier,
    ContextualAdvantageModifier,
    ContextualCriticalModifier,
    ContextualAutoHitModifier,
    ContextualSizeModifier,
    ContextualDamageTypeModifier,
    ResistanceModifier,
    ContextualResistanceModifier,
    ResistanceStatus
)

In [3]:
class Entity(BaseBlock):
    """ Base class for dnd entities in the game it acts as container for blocks and implements common functionalities that
    require interactions between blocks """
    
    name: str = Field(default="Entity")
    ability_scores: AbilityScores = Field(default_factory=lambda: AbilityScores.create(source_entity_uuid=uuid4()))
    skill_set: SkillSet = Field(default_factory=lambda: SkillSet.create(source_entity_uuid=uuid4()))
    saving_throws: SavingThrowSet = Field(default_factory=lambda: SavingThrowSet.create(source_entity_uuid=uuid4()))
    health: Health = Field(default_factory=lambda: Health.create(source_entity_uuid=uuid4()))
    equipped: Equipped = Field(default_factory=lambda: Equipped.create(source_entity_uuid=uuid4()))
    speed: Speed = Field(default_factory=lambda: Speed.create(source_entity_uuid=uuid4()))
    action_economy: ActionEconomy = Field(default_factory=lambda: ActionEconomy.create(source_entity_uuid=uuid4()))
    proficiency_bonus: ModifiableValue = Field(default_factory=lambda: ModifiableValue.create(source_entity_uuid=uuid4(),value_name="proficiency_bonus",base_value=2))
    @classmethod
    def create(cls, source_entity_uuid: UUID, name: str = "Entity") -> 'Entity':
        """
        Create a new Entity instance with the given parameters. All sub-blocks will share
        the same source_entity_uuid as the entity itself.

        Args:
            source_entity_uuid (UUID): The UUID that will be used as both the entity's UUID and source_entity_uuid
            name (str): The name of the entity. Defaults to "Entity"

        Returns:
            Entity: The newly created Entity instance
        """
        return cls(
            uuid=source_entity_uuid,
            source_entity_uuid=source_entity_uuid,
            name=name)
            
    



In [4]:
#static advantage modifier


In [5]:
goblin = Entity.create(source_entity_uuid=uuid4(),name="Goblin")
skeleton = Entity.create(source_entity_uuid=uuid4(),name="Skeleton")
print(goblin.source_entity_uuid)
print(goblin.uuid)
print(skeleton.source_entity_uuid)
print(skeleton.uuid)
goblin.set_target_entity(goblin.uuid)


f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
04963af0-b1ef-4219-8831-9c9b6c149e09
04963af0-b1ef-4219-8831-9c9b6c149e09


In [6]:
from dnd.modifiers import ContextualModifier


self_advantage = AdvantageModifier(name="self_advantage",source_entity_uuid=goblin.uuid,target_entity_uuid=goblin.uuid,value=AdvantageStatus.ADVANTAGE)


def fear_of_skeleton(source_entity: UUID, target_entity: Optional[UUID], context: Optional[Dict[str, Any]] = None) -> AdvantageModifier:
    nonemodifier= AdvantageModifier(name="fear_of_skeleton",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=AdvantageStatus.NONE)
    if target_entity is None:
        return nonemodifier

    target_entity_name = Entity.get(target_entity)
    if target_entity_name is None:
        return nonemodifier
    if target_entity_name.name == "Skeleton":
        return AdvantageModifier(name="fear_of_skeleton",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=AdvantageStatus.DISADVANTAGE)
    else:


        return nonemodifier


ifskeleton_modifier= ContextualAdvantageModifier(name="contextual_fear_of_skeleton",source_entity_uuid=goblin.uuid,target_entity_uuid=goblin.uuid,callable=fear_of_skeleton)



In [7]:
#try to compute the modifieable value for a skill roll
# gotta get a) the proficiency bonus b) the ability modifier c) the skill bonus
# we want to keep them as modifiable value and combine them together
assert isinstance(goblin, Entity)
proficiency_modifier = goblin.proficiency_bonus
extracted_skill = goblin.skill_set.get_skill("acrobatics")
skill_modifier = extracted_skill.skill_bonus
extracted_ability = goblin.ability_scores.get_ability(extracted_skill.ability)
ability_modifier = extracted_ability.ability_score
#oprint all source entity uuids
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")


Proficiency Bonus: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
Skill Bonus: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
Ability Modifier: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006


In [8]:
ability_modifier.normalized_score

0

In [9]:
ability_modifier.global_normalizer

True

In [10]:
ability_modifier.score_normalizer

<function dnd.blocks.ability_score_normalizer(score: int) -> int>

In [11]:
numerical_modifier_dict = ability_modifier.self_static.value_modifiers
numerical_modifier = numerical_modifier_dict[list(ability_modifier.self_static.value_modifiers.keys())[0]]

In [12]:
print(numerical_modifier.value)
print(numerical_modifier.normalized_value)
print(numerical_modifier.score_normalizer)

10
0
<function ability_score_normalizer at 0x10752b380>


In [13]:
numerical_modifier = ability_modifier.self_static.value_modifiers[list(ability_modifier.self_static.value_modifiers.keys())[0]]



In [14]:
normalizer =numerical_modifier.score_normalizer
normalizer

<function dnd.blocks.ability_score_normalizer(score: int) -> int>

In [15]:
normalizer(19)

4

In [16]:
proficiency_modifier.self_static.target_entity_uuid


UUID('f3ea5e8f-1788-4bfd-83a9-4beb00cfe006')

In [17]:
skill_modifier.self_static.target_entity_uuid

UUID('f3ea5e8f-1788-4bfd-83a9-4beb00cfe006')

In [18]:
from typing import Tuple
def skill_merge_status(proficiency_bonus: ModifiableValue, skill_bonus: ModifiableValue, ability_modifier: ModifiableValue) -> Tuple[AdvantageStatus, CriticalStatus, AutoHitStatus, int, int]:
    merged_value = proficiency_bonus.combine_values([skill_bonus, ability_modifier])          

    return merged_value.advantage,merged_value.critical,merged_value.auto_hit,merged_value.normalized_score,merged_value.score
goblin.set_target_entity(goblin.uuid)

advantage_status = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status)
proficiency_modifier.self_static.add_advantage_modifier(self_advantage)
proficiency_modifier.self_contextual.add_advantage_modifier(ifskeleton_modifier)
advantage_status_post = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post)
goblin.set_target_entity(skeleton.uuid)
advantage_status_post_target = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post_target)




Validating others source id f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 against self source f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 for modifer  with name proficiency_bonus and uuid e2d22658-5b9a-4a2b-8db6-4f741b99579c and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid 48be0d52-0564-4c37-956a-11790f97a962
Validating others source id f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 against self source f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 for modifer  with name proficiency_bonus and uuid e2d22658-5b9a-4a2b-8db6-4f741b99579c and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Ability Score and uuid 71192eee-dce4-49a6-aebb-7c83063db0cf
(<AdvantageStatus.NONE: 'None'>, <CriticalStatus.NONE: 'None'>, <AutoHitStatus.NONE: 'None'>, 12, 12)
Validating others source id f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 against self source f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 for modifer  with name proficiency_bon

In [19]:
ifskeleton_modifier.uuid
proficiency_modifier.remove_modifier(ifskeleton_modifier.uuid)
advantage_status_post_remove = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post_remove)

Validating others source id f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 against self source f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 for modifer  with name proficiency_bonus and uuid e2d22658-5b9a-4a2b-8db6-4f741b99579c and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid 48be0d52-0564-4c37-956a-11790f97a962
Validating others source id f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 against self source f3ea5e8f-1788-4bfd-83a9-4beb00cfe006 for modifer  with name proficiency_bonus and uuid e2d22658-5b9a-4a2b-8db6-4f741b99579c and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Ability Score and uuid 71192eee-dce4-49a6-aebb-7c83063db0cf
(<AdvantageStatus.ADVANTAGE: 'Advantage'>, <CriticalStatus.NONE: 'None'>, <AutoHitStatus.NONE: 'None'>, 12, 12)


In [20]:
#formatetd print of the bonus
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")



Proficiency Bonus: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
Skill Bonus: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006
Ability Modifier: f3ea5e8f-1788-4bfd-83a9-4beb00cfe006


In [21]:
proficiency_modifier.normalized_score

2

In [22]:
from typing import Tuple, Optional
import random

def roll_d20(advantage_status: AdvantageStatus) -> Tuple[int,Optional[int]]:
        num_rolls = 1 if advantage_status == AdvantageStatus.NONE else 2
        rolls = [random.randint(1,20) for _ in range(num_rolls)]
        if advantage_status == AdvantageStatus.ADVANTAGE:
            return max(rolls), min(rolls)
        elif advantage_status == AdvantageStatus.DISADVANTAGE:
            return min(rolls), max(rolls)
        else:
            return rolls[0], None
        
        
def skill_roll_bonus(skill_name: SkillName, entity: Entity) -> DiceRoll:
    extracted_skill = entity.skill_set.get_skill(skill_name)
    skill_modifier = extracted_skill.skill_bonus
    extracted_ability = entity.ability_scores.get_ability(extracted_skill.ability)
    ability_modifier = extracted_ability.ability_score



        

       
    proficiency_modifier = entity.proficiency_bonus

    

    return DiceRoll(
        roll_type=RollType.SKILL,
        results=,
        source_entity_uuid=entity.uuid,
        target_entity_uuid=None,
        attack_outcome=None
    )



SyntaxError: expected argument value expression (2776065904.py, line 32)